<img src="https://heig-vd.ch/docs/default-source/doc-global-newsletter/2020-slim.svg" alt="HEIG-VD Logo" width="100"/>

# Cours TAL - Laboratoire 6
# Trois méthodes de désambiguïsation lexicale

**Objectif**

L'objectif de ce laboratoire est d'implémenter et de comparer plusieurs méthodes de désambiguïsation lexicale (en anglais, *Word Sense Disambiguation* ou WSD).  Vous utiliserez un corpus avec plusieurs milliers de phrases, chaque phrase contenant une occurrence du mot anglais *interest* annotée avec le sens que ce mot possède dans la phrase respective.  Les trois méthodes sont les suivantes (elles seront détaillées par la suite) :

* Algorithme de Lesk simplifié.
* Utilisation de word2vec.
* Classification supervisée utilisant des traits lexicaux.

Les deux premières méthodes n'utilisent pas l'apprentissage automatique.  Elles fonctionnent selon le même principe : comparer le contexte d'une occurrence de *interest* avec chacune des définitions des sens (*synsets*) et choisir la définition la plus proche du contexte.  L'algorithme de Lesk définit la proximité comme le nombre de mots en commun, alors que word2vec la calcule comme la similarité de vecteurs.  La dernière méthode vise à classifier les occurrences de *interest*, les sens étant les classes, et les attributs étant les mots du contexte (apprentissage supervisé).

## 1. Analyse des données

Téléchargez le corpus *interest* depuis le [site du Prof. Ted Pedersen](http://www.d.umn.edu/~tpederse/data.html) (il se trouve en bas de sa page web).  Téléchargez l'archive ZIP marquée *original format without POS tags* et extrayez le fichier `interest-original.txt`.  Téléchargez également le fichier `README.int.txt` indiqué à la ligne au-dessus. Veuillez répondre brièvement aux questions suivantes :

a. Quelles sont les URL du fichier ZIP et celle du fichier `README.int.txt` ?

b. Quel est le format du fichier `interest-original.txt` et comment sont annotés les sens de *interest* ?

c. Est-ce qu'il y a aussi des occurrences au pluriel (*interests*) à traite ?

d. Comment sont annotées les phrases qui contiennent plusieurs occurrences du mot *interest* ?

**1a.** Quelles sont les URL du fichier ZIP et celle du fichier `README.int.txt` ?

In [2]:
# ZIP: https://www.d.umn.edu/~tpederse/Data/interest-original.nopos.tar.gz
# README.int.txt: https://www.d.umn.edu/~tpederse/Data/README.int.txt

**1b.** Quel est le format du fichier `interest-original.txt` et comment sont annotés les sens de *interest* ?


In [3]:
# Le fichier est un corpus composé de 2369 phrases extraites du Wall Street Journal (Penn Treebank).
# Chaque phrase est séparée par une ligne contenant uniquement $$.
# Chaque phrase contient exactement une occurrence annotée du mot interest ou interests (au pluriel).
# Les sens sont annotés manuellement selon 6 sens définis par le Longman Dictionary of Contemporary English (LDOCE) :
# interest_1 : attention ou curiosité
# interest_2 : qualité de susciter l'attention
# interest_3 : activité à laquelle on porte attention
# interest_4 : avantage ou intérêt personnel
# interest_5 : part d'une entreprise ou affaire
# interest_6 : intérêt au sens financier (argent)
# L’annotation suit ce format : interest_6/NN (mot + numéro de sens + étiquette morpho-syntaxique POS)

**1c.** Est-ce qu'il y a aussi des occurrences au pluriel (*interests*) à traite ?


In [4]:
# Oui. Le corpus contient des occurrences au pluriel annotées de la même façon : interests_5/NNS, interests_4/NNS, etc.

**1d.** Comment sont annotées les phrases qui contiennent plusieurs occurrences du mot *interest* ?

In [5]:
# Chaque occurrence unique dans une phrase est annotée individuellement avec son propre numéro de sens (interest_1, interest_5, etc.).
# Ces phrases restent dans une seule ligne séparée par $$ comme les autres.

**1e.** D'après le fichier `README.int.txt`, quelles sont les définitions des six sens de *interest* annotés dans les données et quelles sont leurs fréquences ? Vous pouvez copier/coller l'extrait de `README`ici.

In [6]:
# Veuillez répondre ici (en commentaire) à la question.
# Sense 1 =  361 occurrences (15%) - readiness to give attention  
# Sense 2 =   11 occurrences (01%) - quality of causing attention to be given to  
# Sense 3 =   66 occurrences (03%) - activity, etc. that one gives attention to  
# Sense 4 =  178 occurrences (08%) - advantage, advancement or favor  
# Sense 5 =  500 occurrences (21%) - a share in a company or business  
# Sense 6 = 1252 occurrences (53%) - money paid for the use of money  

**1f.** De quel dictionnaire viennent les sens précédents ? Où peut-on le consulter en ligne ?  Veuillez aligner les définitions du dictionnaire avec les six sens annotés en écrivant par exemple `Sense 3 = "an activity that you enjoy doing or a subject that you enjoy studying"`.

In [7]:
# Veuillez répondre ici (en commentaire) à la question.
# Les sens viennent du Longman Dictionary of Contemporary English (LDOCE).
# Il est mentionné dans le README : « the six non-idiomatic noun senses of "interest" defined in the electronic version of the first edition of Longman's Dictionary of Contemporary English ».
# On peut consulter une version en ligne ici : https://www.ldoceonline.com/

In [8]:
# Alignement des définitions du dictionnaire (exemples reformatés) :
#    Sense 1 = "readiness to give attention"
#        e.g., "He listened with great interest."
#    Sense 2 = "quality of causing attention to be given to"
#        e.g., "The story is of little interest."
#    Sense 3 = "an activity that you enjoy doing or subject that you enjoy studying"
#        e.g., "My main interests are photography and hiking."
#    Sense 4 = "advantage, benefit or self-serving motive"
#        e.g., "She acted in her own best interest."
#    Sense 5 = "a share in a business or financial involvement"
#        e.g., "He bought a controlling interest in the company."
#    Sense 6 = "money paid for the use of money"
#        e.g., "The bank charges 5% interest."

**1g.** En consultant [WordNet en ligne](http://wordnetweb.princeton.edu/perl/webwn), trouvez les définitions des synsets  pour le **nom commun** *interest*.  Combien de synsets y a-t-il ?  Veuillez indiquer comme avant la **définition** de chaque synset pour chacun des six sens ci-dessus (au besoin, fusionner ou ignorer des synsets).

In [9]:
import nltk

nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/massimo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [10]:
from nltk.corpus import wordnet as wn

synsets = wn.synsets("interest", pos=wn.NOUN)
print(f"Nombre de synsets pour 'interest' (nom) : {len(synsets)}")
for i, syn in enumerate(synsets):
    print(f"Synset {i+1} : {syn.name()}")
    print(f"Définition : {syn.definition()}")
    print(f"Lemmes : {syn.lemma_names()}")
    print(f"Exemples : {syn.examples()}")
    print(f"=================================")


Nombre de synsets pour 'interest' (nom) : 7
Synset 1 : interest.n.01
Définition : a sense of concern with and curiosity about someone or something
Lemmes : ['interest', 'involvement']
Exemples : ['an interest in music']
Synset 2 : sake.n.01
Définition : a reason for wanting something done
Lemmes : ['sake', 'interest']
Exemples : ['for your sake', 'died for the sake of his country', 'in the interest of safety', 'in the common interest']
Synset 3 : interest.n.03
Définition : the power of attracting or holding one's attention (because it is unusual or exciting etc.)
Lemmes : ['interest', 'interestingness']
Exemples : ['they said nothing of great interest', 'primary colors can add interest to a room']
Synset 4 : interest.n.04
Définition : a fixed charge for borrowing money; usually a percentage of the amount borrowed
Lemmes : ['interest']
Exemples : ['how much interest do you pay on your mortgage?']
Synset 5 : interest.n.05
Définition : (law) a right or legal share of something; a financia

- **Sense 1** = "readiness to give attention"  
  → WordNet : *interest.n.01*  
  → Définition : *a sense of concern with and curiosity about someone or something*

- **Sense 2** = "quality of causing attention to be given to"  
  → WordNet : *interest.n.03*  
  → Définition : *the power of attracting or holding one's attention (because it is unusual or exciting etc.)*

- **Sense 3** = "activity, etc. that one gives attention to"  
  → WordNet : *interest.n.07*  
  → Définition : *a diversion that occupies one's time and thoughts (usually pleasantly)*

- **Sense 4** = "advantage, advancement or favor"  
  → WordNet : *sake.n.01*  
  → Définition : *a reason for wanting something done*

- **Sense 5** = "a share in a company or business"  
  → WordNet : *interest.n.05*  
  → Définition : *(law) a right or legal share of something; a financial involvement with something*

- **Sense 6** = "money paid for the use of money"  
  → WordNet : *interest.n.04*  
  → Définition : *a fixed charge for borrowing money; usually a percentage of the amount borrowed*


**1h.** Définissez (manuellement, ou avec quelques lignes de code) une liste nommée `senses1` avec les mots des définitions du README, en supprimant les stopwords (p.ex. les mots < 4 lettres).  Affichez la liste.

In [11]:
import nltk
#nltk.download('stopwords')

In [12]:
from random import randrange

In [13]:
# Veuillez répondre ici à la question et créer la variable 'senses1' (liste de 6 listes de chaînes).
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

definitions = [
    "readiness to give attention",
    "quality of causing attention to be given to",
    "activity, etc. that one gives attention to",
    "advantage, advancement or favor",
    "a share in a company or business",
    "money paid for the use of money"
]

stop_words = set(stopwords.words("english")) | set(string.punctuation)
senses1 = []
for definition in definitions:
    tokens = word_tokenize(definition.lower())
    filtered = [w for w in tokens if w.isalpha() and len(w) >= 4 and w not in stop_words]
    unique_words = list(set(filtered))
    senses1.append(unique_words)

print(senses1)


[['readiness', 'attention', 'give'], ['given', 'attention', 'causing', 'quality'], ['attention', 'activity', 'gives'], ['advantage', 'advancement', 'favor'], ['business', 'company', 'share'], ['money', 'paid']]


**1i.** En combinant les définitions obtenues aux points (4) et (5) ci-dessus, construisez une liste nommée `senses2` avec pour chacun des sens de *interest* une liste de **mots-clés** correspondants.  Vous pouvez concaténer les définitions, puis écrire des instructions en Python pour extraire les mots (uniques).  Respectez l'ordre des sens données par `README`, et à la fin affichez `senses2`.

In [14]:
# Veuillez répondre ici à la question et créer la variable 'senses2' (liste de 6 listes de chaînes).
senses2 = [
    ["attention", "curiosity", "concern", "readiness"],
    ["attention", "attracting", "excit£ing", "noteworthy", "unusual"],
    ["hobby", "activity", "diversion", "recreation", "pastime"],
    ["benefit", "advantage", "favor", "gain", "profit"],
    ["share", "ownership", "stake", "equity", "business", "company"],
    ["money", "loan", "borrowing", "charge", "percentage", "payment"]
]

print(senses2)


[['attention', 'curiosity', 'concern', 'readiness'], ['attention', 'attracting', 'excit£ing', 'noteworthy', 'unusual'], ['hobby', 'activity', 'diversion', 'recreation', 'pastime'], ['benefit', 'advantage', 'favor', 'gain', 'profit'], ['share', 'ownership', 'stake', 'equity', 'business', 'company'], ['money', 'loan', 'borrowing', 'charge', 'percentage', 'payment']]


**1j.** Chargez les données depuis `interest-original.txt` dans une liste appelée `sentences` qui contient pour chaque phrase la liste des mots (sans les séparateurs *$$* et *===...*).  Ces phrases sont-elles déjà tokenisées en mots ?  Sinon, faites-le.  À ce stade, ne modifiez pas encore les occurrences annotées *interest(s)\_X*.  Comptez le nombre total de phrases et affichez-en trois au hasard.

In [15]:
# Veuillez répondre ici à la question.
with open("interest-original.txt", encoding="utf-8") as f:
    data = f.read()

sentences = [s.strip().split() for s in data.split("$$") if s.strip()]

print("Il y a {} phrases.\nEn voici 3 au hasard :".format(len(sentences)))
from random import sample
for s in sample(sentences, 3):
    print(" ".join(s))

Il y a 2368 phrases.
En voici 3 au hasard :
but he said yesterday that the court had gone `` into the matter with great care and the public interest_4 has been served '' by the sentence .
====================================== currently , mr. merksamer owns 20 % of the company ; l.j. hooker acquired its 80 % interest_5 in the firm in may 1986 .
the zero-coupon subordinated notes have no periodic interest_6 payments .


## 2. Algorithme de Lesk simplifié

**2a.** Définissez une fonction `wsd_lesk(senses, sentence)` qui prend deux arguments : une liste de listes de mots-clés (comme `senses1` et `senses2` ci-dessus) et une phrase avec une occurrence annotée de *interest* ou *interests*, et qui retourne l'index du sens le plus probable (entre 1 et 6) selon l'algorithme de Lesk.  Cet algorithme choisit le sens qui a le maximum de mots en commun avec le contexte de *interest*.  Vous pouvez choisir vous-mêmes la taille de ce voisinage (`window_size`).  En cas d'égalité entre deux sens, tirer la réponse au sort.

In [16]:
# Veuillez répondre ici à la question.
from nltk.corpus import stopwords
import random

def wsd_lesk(senses, sentence, window_size=8):
    stop_words = set(stopwords.words('english'))
    
    # Trouver l'index de "interest" annoté avec un sens ou "interests"
    index_interest = next(i for i, word in enumerate(sentence) if (word.startswith("interest_") or word.startswith("interests_")))
    
    # Nettoyage du contexte (voisinage gauche + droite)
    context = sentence[max(0, index_interest - window_size): index_interest] + \
              sentence[index_interest + 1: index_interest + 1 + window_size]
    
    context_clean = [w.lower() for w in context if w.lower() not in stop_words and len(w) > 3]
    
    # Compter le chevauchement avec chaque sens
    overlaps = []
    for sense_keywords in senses:
        overlap = len(set(context_clean) & set(sense_keywords))
        overlaps.append(overlap)
    
    # Résoudre égalités au hasard
    max_overlap = max(overlaps)
    candidates = [i for i, v in enumerate(overlaps) if v == max_overlap]
    
    return random.choice(candidates) + 1  # retour 1 à 6 (pas 0 à 5)


**2b.** Définissez maintenant une fonction `evaluate_wsd(fct_name, senses, sentences)` qui prend en paramètre le nom de la méthode de similarité (pour commencer : `wsd_lesk`) ainsi que la liste des mots-clés par sens, et la liste de phrases, et qui retourne le score de la méthode de similarité.  Ce score sera tout simplement le pourcentage de réponses correctes (sens trouvé identique au sens annoté).

In [17]:
# Veuillez répondre ici à la question.
def evaluate_wsd(fct_name, senses, sentences, window_size=8):
    correct = 0
    total = 0
    
    for sentence in sentences:
        # Extraire le vrai sens
        target = next((w for w in sentence if (w.startswith("interest_") or w.startswith("interests_")) ), None)
        if not target:
            continue
        true_sense = int(target.split("_")[1].split("/")[0])
        
        predicted_sense = fct_name(senses, sentence, window_size)
        
        if predicted_sense == true_sense:
            correct += 1
        total += 1
    
    return 100 * correct / total


**2c.** En fixant au mieux la taille de la fenêtre autour de *interest*, quel est le meilleur score de la méthode de Lesk simplifiée ?  Quelle liste de sens conduit à de meilleurs scores, `senses1` ou `senses2` ?

In [18]:
# Veuillez répondre ici à la question.

senses = [senses1, senses2]

# Veuillez répondre ici à la question.
sens = [senses1, senses2]
for i, s in enumerate(sens, 0):
    print(f"\nÉvaluation avec senses{i+1} :")
    for ws in (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20):
        score = evaluate_wsd(
            lambda sens, sent, _: wsd_lesk(sens, sent, window_size=ws),
            sens[i], sentences)
        print(f"fenêtre {ws} → {score:.2f}%")




Évaluation avec senses1 :
fenêtre 1 → 16.26%
fenêtre 2 → 16.09%
fenêtre 3 → 17.53%
fenêtre 4 → 17.78%
fenêtre 5 → 19.93%
fenêtre 6 → 19.72%
fenêtre 7 → 18.83%
fenêtre 8 → 19.13%
fenêtre 9 → 20.40%
fenêtre 10 → 20.65%
fenêtre 11 → 20.95%
fenêtre 12 → 21.16%
fenêtre 13 → 20.61%
fenêtre 14 → 19.38%
fenêtre 15 → 20.73%
fenêtre 16 → 21.58%
fenêtre 17 → 20.40%
fenêtre 18 → 21.54%
fenêtre 19 → 20.57%
fenêtre 20 → 22.09%

Évaluation avec senses2 :
fenêtre 1 → 18.58%
fenêtre 2 → 18.96%
fenêtre 3 → 19.59%
fenêtre 4 → 19.64%
fenêtre 5 → 21.16%
fenêtre 6 → 22.26%
fenêtre 7 → 22.59%
fenêtre 8 → 23.02%
fenêtre 9 → 23.82%
fenêtre 10 → 23.06%
fenêtre 11 → 23.35%
fenêtre 12 → 23.69%
fenêtre 13 → 24.11%
fenêtre 14 → 23.10%
fenêtre 15 → 23.02%
fenêtre 16 → 23.52%
fenêtre 17 → 24.07%
fenêtre 18 → 24.07%
fenêtre 19 → 24.49%
fenêtre 20 → 24.83%


## 3. Utilisation de word2vec pour la similarité contexte vs. synset

**3a.** En réutilisant une partie du code de `wsd_lesk`, veuillez maintenant définir une fonction `wsd_word2vec(senses, sentence)` qui choisit le sens en utilisant la similarité **word2vec** étudiée dans le labo précédent. 
* Vous pouvez chercher dans la [documentation des KeyedVectors](https://radimrehurek.com/gensim/models/keyedvectors.html) comment calculer directement la similarité entre deux listes de mots.
* Comme `wsd_lesk`, la nouvelle fonction `wsd_word2vec` prend en argument une liste de listes de mots-clés par sens (comme `senses1` et `senses2` ci-dessus), et une phrase avec une occurrence annotée de *interest* ou *interests*.
* La fonction retourne le numéro du sens le plus probable selon la similarité word2vec entre les mots du sens et ceux du voisinage de *interest*.  En cas d'égalité, tirer le sens au sort.
* Vous pouvez régler la taille du voisinage (`window_size`) par l'expérimentation.  

In [19]:
from gensim import downloader as api
w2v_vectors = api.load("word2vec-google-news-300")

In [20]:
import gensim
from gensim.models import KeyedVectors
path_to_model = "~/gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz" # à adapter
wv_model = gensim.models.KeyedVectors.load_word2vec_format(path_to_model, binary=True)  # C bin format

In [21]:
from nltk.corpus import stopwords
import random

def wsd_word2vec(senses, sentence, window_size=8,
                 target_prefixes=("interest_", "interests_")):
    """
    Choisit le sens d’une occurrence de « interest / interests » par similarité Word2Vec.
    Retourne un entier entre 1 et len(senses).
    Compatible avec evaluate_wsd(senses, sentence, window_size=...).
    """
    # 1. index du mot annoté
    idx = next(i for i, w in enumerate(sentence)
               if any(w.startswith(pref) for pref in target_prefixes))

    # 2. contexte (on enlève le mot cible)
    left  = sentence[max(0, idx - window_size): idx]
    right = sentence[idx + 1: idx + 1 + window_size]
    raw_context = left + right

    # 3. nettoyage (stop-words)
    sw = set(stopwords.words("english"))
    context = [w.lower() for w in raw_context
               if w.lower() not in sw and len(w) > 3 and w in wv_model]

    if not context: 
        # fallback aléatoire pour si le contexte est vide après nettoyage des stop-words
        return random.randint(1, len(senses))

    # 4. similarité contexte ↔ signature pour chaque sens
    sims = []
    for sig in senses:
        sig = [w for w in sig if w in wv_model]
        sims.append(
            wv_model.n_similarity(context, sig) if sig else -1)

    # 5. meilleur score (tirage en cas d’égalité)
    best = [i for i, s in enumerate(sims) if s == max(sims)]
    return random.choice(best) + 1


**3b.** Appliquez maintenant la même méthode `evaluate_wsd` avec la fonction `wsd_word2vec` (en cherchant une bonne valeur de la taille de la fenêtre) et affichez le score de la similarité word2vec.  Comment se compare-t-il avec le score précédent (Lesk) ?

In [22]:
# Veuillez répondre ici à la question.
sens = [senses1, senses2]
for i, s in enumerate(sens, 0):
    print(f"\nÉvaluation avec senses{i+1} :")
    for ws in (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20):
        score = evaluate_wsd(
            lambda sens, sent, _: wsd_word2vec(sens, sent, window_size=ws),
            sens[i], sentences)
        print(f"fenêtre {ws} → {score:.2f}%")



Évaluation avec senses1 :
fenêtre 1 → 24.54%
fenêtre 2 → 29.39%
fenêtre 3 → 32.94%
fenêtre 4 → 37.04%
fenêtre 5 → 37.33%
fenêtre 6 → 37.75%
fenêtre 7 → 38.39%
fenêtre 8 → 38.68%
fenêtre 9 → 39.61%
fenêtre 10 → 40.03%
fenêtre 11 → 41.05%
fenêtre 12 → 41.43%
fenêtre 13 → 40.75%
fenêtre 14 → 40.62%
fenêtre 15 → 40.20%
fenêtre 16 → 40.50%
fenêtre 17 → 40.62%
fenêtre 18 → 40.54%
fenêtre 19 → 40.41%
fenêtre 20 → 40.46%

Évaluation avec senses2 :
fenêtre 1 → 55.91%
fenêtre 2 → 57.31%
fenêtre 3 → 58.36%
fenêtre 4 → 59.76%
fenêtre 5 → 61.23%
fenêtre 6 → 61.66%
fenêtre 7 → 61.78%
fenêtre 8 → 61.57%
fenêtre 9 → 61.70%
fenêtre 10 → 61.70%
fenêtre 11 → 62.20%
fenêtre 12 → 62.29%
fenêtre 13 → 62.42%
fenêtre 14 → 62.25%
fenêtre 15 → 62.04%
fenêtre 16 → 62.16%
fenêtre 17 → 61.70%
fenêtre 18 → 62.04%
fenêtre 19 → 62.12%
fenêtre 20 → 62.25%


## 4. Classification supervisée avec des traits lexicaux
Vous entraînerez maintenant des classifieurs pour prédire le sens d'une occurrence dans une phrase.  Le premier but sera de transformer chaque phrase en un ensemble d'attributs pour formater les données en vue des expériences de classification.

Veuillez utiliser le classifieur `NaiveBayesClassifier` fourni par NLTK.  Le mode d'emploi se trouve dans le [Chapitre 6, sections 1.1-1.3](https://www.nltk.org/book/ch06.html) du livre NLTK.  Consultez-le attentivement pour trouver comment formater les données.  De plus, il faudra séparer les données en sous-ensembles d'entraînement et de test.

On vous propose de nommer les attributs `word-k`, ..., `word-2`, `word-1`, `word+1`, `word+2`, ..., `word+k` (fenêtre de taille `2*k` autour de *interest*).  Leurs valeurs sont les mots observés aux emplacements respectifs, ou `NONE` si la position dépasse l'étendue de la phrase.  Vous ajouterez un attribut nommé `word0` qui est l'occurrence du mot *interest* au singulier ou au pluriel.  

Pour chaque occurrence de *interest*, vous devrez donc créer la représentation suivante (où `6` est le numéro du sens, essentiel pour l'entraînement, mais à cacher lors de l'évaluation) :
```
[{'word-1': 'in', 'word+1': 'rates', 'word-2': 'declines', 'word+2': 'NONE', 'word0': 'interest'}, 6]
```

**4a.** En partant de la liste des phrases appelée `sentences` préparée plus haut, veuillez générer la liste avec toutes les représentation, appelée `items_with_features`.  Vous pouvez vous aider du livre NLTK.

In [23]:
# Veuillez répondre ici à la question.
items_with_features = []


def extract_features(input_sentences, k):
    result_features = []

    for sentence in input_sentences:
        
        for i, word in enumerate(sentence):
            if word.startswith('interest'):
                features = {}
                sens = ""
                
                try:
                    if '_' in word: # We got a match !
                        (word0, sens) = word.split('_')
                        features['word0'] = word0
                        
                        for depth in range(1, k + 1):
                            # Left context
                            if i - depth >= 0:
                                features[f'word-{depth}'] = sentence[i - depth].lower()
                            else:
                                features[f'word-{depth}'] = 'NONE'
                            
                            # Right context
                            if i + depth < len(sentence):
                                features[f'word+{depth}'] = sentence[i + depth].lower()
                            else:
                                features[f'word+{depth}'] = 'NONE'
                            
        
                except Exception as e:
                    print(f"Error processing word '{word}' in {sentence}: {e}")
                    
                result_features.append([features, sens])
                break
            
    return result_features
                
items_with_features = extract_features(sentences, 2)

print(f"Nombre total d'instances: {len(items_with_features)}")
print("Exemples d'instances:")
for i in range(min(3, len(items_with_features))):
    print(items_with_features[i])

Nombre total d'instances: 2368
Exemples d'instances:
[{'word0': 'interest', 'word-1': 'in', 'word+1': 'rates', 'word-2': 'declines', 'word+2': '.'}, '6']
[{'word0': 'interest', 'word-1': 'declining', 'word+1': 'rates', 'word-2': 'indicate', 'word+2': 'because'}, '6']
[{'word0': 'interest', 'word-1': 'short-term', 'word+1': 'rates', 'word-2': 'in', 'word+2': '.'}, '6']


**4b.** Veuillez séparer les données aléatoirement en 80% pour l'entraînement et 20%  pour l'évaluation.  Veuillez faire une division stratifiée : les deux sous-ensembles doivent contenir les mêmes proportions de sens que l'ensemble de départ.  Ils seront appelés `iwf_train` et `iwf_test`.

In [24]:
from sklearn.model_selection import train_test_split

def split_data_train_test(items_with_features, test_size=0.2):

    # Cleanup empty features or senses
    items_with_features = [item for item in items_with_features if item[1] != '']
    print(f"Cleaned data, new length: {len(items_with_features)}")


    X = [item[0] for item in items_with_features]
    y = [item[1] for item in items_with_features]
    
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=42, stratify=y
    )
    
    return list(zip(X_train, y_train)), list(zip(X_test, y_test))

iwf_train, iwf_test = split_data_train_test(items_with_features, test_size=0.2)


print(iwf_test[:3])

Cleaned data, new length: 2367
[({'word0': 'interest', 'word-1': 'short', 'word+1': 'dropped', 'word-2': 'that', 'word+2': 'to'}, '5'), ({'word0': 'interest', 'word-1': '}', 'word+1': 'rates', 'word-2': 'NONE', 'word+2': ','}, '6'), ({'word0': 'interests', 'word-1': 'minority', 'word+1': 'rose', 'word-2': 'to', 'word+2': 'to'}, '5')]


In [25]:
print("Total :", len(items_with_features))
print("Train :", len(iwf_train), "| Test :", len(iwf_test))
print("Sample train:", iwf_train[0])
print("Sample test :", iwf_test[0])

Total : 2368
Train : 1893 | Test : 474
Sample train: ({'word0': 'interest', 'word-1': 'in', 'word+1': 'rates', 'word-2': 'dips', 'word+2': '.'}, '6')
Sample test : ({'word0': 'interest', 'word-1': 'short', 'word+1': 'dropped', 'word-2': 'that', 'word+2': 'to'}, '5')


**4c.** Veuillez créer une instance de `NaiveBayesClassifier`, l'entraîner sur `iwf_train` et la tester sur `iwf_test` (voir la documentation NLTK).  En expérimentant avec différentes largeurs de fenêtres, quel est le meilleur score que vous obtenez (avec la fonction `accuracy` de NLTK) sur l'ensemble de test ?  Comment se compare-t-il avec les précédents ?

In [28]:
from nltk.classify import naivebayes 
# Veuillez répondre ici à la question.


# Dictionnaire pour stocker les résultats
results = {}

# En utilisant le trainset de base, voici les résultats :
def classify_with_naive_bayes(tr_d, te_d, k=2):
    
    # Entraînement du classificateur Naive Bayes
    c = naivebayes.NaiveBayesClassifier.train(tr_d)
    
    # Calcul de l'accuracy sur les ensembles d'entraînement et de test
    train_acc = nltk.classify.accuracy(c, tr_d)
    test_acc = nltk.classify.accuracy(c, te_d)

    print(f"Classifier accuracy (train) with k={k}:", train_acc)
    print(f"Classifier accuracy (test) with k={k}:", test_acc)
    
    return c, train_acc, test_acc

# Classification avec Naive Bayes
classifier, train_accuracy, test_accuracy = classify_with_naive_bayes(iwf_train, iwf_test)

# Garde en mémoire le résultat pour la suite
results['naive_bayes_k2'] = {
    'train': train_accuracy,
    'test': test_accuracy,
    'window_size': 2
}

Classifier accuracy (train) with k=2: 0.9614368726888537
Classifier accuracy (test) with k=2: 0.8945147679324894


In [29]:
for k in range(1, 7):
    if k == 2: # On a déjà testé k=2
        continue

    print(f"\nTesting with k={k}...")
    
    # Extraire les caractéristiques avec le nouveau k
    iwf_train_k, iwf_test_k = split_data_train_test(extract_features(sentences, k), test_size=0.2)
    
    classifier_k, train_accuracy_k, test_accuracy_k = classify_with_naive_bayes(iwf_train_k, iwf_test_k, k)
    
    # Stocker dans le dictionnaire
    results[f'naive_bayes_k{k}'] = {
        'train': train_accuracy_k,
        'test': test_accuracy_k,
        'window_size': k
    }

# Afficher le résumé des résultats
print("\n=== RÉSUMÉ DES RÉSULTATS ===")
for method, scores in results.items():
    print(f"{method}: Train={scores['train']:.3f}, Test={scores['test']:.3f}, Window={scores['window_size']}")

# Trouver le meilleur score de test
best_method = max(results.items(), key=lambda x: x[1]['test'])
print(f"\nMeilleur score: {best_method[0]} avec {best_method[1]['test']:.3f} sur le test")


Testing with k=1...
Cleaned data, new length: 2367
Classifier accuracy (train) with k=1: 0.9265715795034337
Classifier accuracy (test) with k=1: 0.890295358649789

Testing with k=3...
Cleaned data, new length: 2367
Classifier accuracy (train) with k=3: 0.9804543053354464
Classifier accuracy (test) with k=3: 0.8924050632911392

Testing with k=4...
Cleaned data, new length: 2367
Classifier accuracy (train) with k=4: 0.9846804014791336
Classifier accuracy (test) with k=4: 0.8881856540084389

Testing with k=5...
Cleaned data, new length: 2367
Classifier accuracy (train) with k=5: 0.9904912836767037
Classifier accuracy (test) with k=5: 0.8860759493670886

Testing with k=6...
Cleaned data, new length: 2367
Classifier accuracy (train) with k=6: 0.993660855784469
Classifier accuracy (test) with k=6: 0.8734177215189873

=== RÉSUMÉ DES RÉSULTATS ===
naive_bayes_k2: Train=0.961, Test=0.895, Window=2
naive_bayes_k1: Train=0.927, Test=0.890, Window=1
naive_bayes_k3: Train=0.980, Test=0.892, Window

In [30]:
best_k = best_method[1]['window_size']
print(f"\nClassification finale avec k={best_k}...")
iwf_train_best, iwf_test_best = split_data_train_test(extract_features(sentences, best_k), test_size=0.2)
best_classifier, best_train_accuracy, best_test_accuracy = classify_with_naive_bayes(iwf_train_best, iwf_test_best)


Classification finale avec k=2...
Cleaned data, new length: 2367
Classifier accuracy (train) with k=2: 0.9614368726888537
Classifier accuracy (test) with k=2: 0.8945147679324894


**4d.** En utilisant la fonction `show_most_informative_features()`, veuillez afficher les attributs les plus informatifs et commenter le résultat.

In [31]:
classifier.show_most_informative_features(20)

Most Informative Features
                   word0 = 'interests'         3 : 1      =     89.1 : 1.0
                  word+1 = 'in'                5 : 6      =     87.6 : 1.0
                  word+1 = 'of'                4 : 6      =     63.2 : 1.0
                  word-1 = 'other'             3 : 6      =     42.3 : 1.0
                  word+2 = '.'                 6 : 1      =     36.0 : 1.0
                  word+2 = ','                 6 : 5      =     28.1 : 1.0
                  word-2 = 'have'              1 : 6      =     19.5 : 1.0
                  word-1 = 'in'                6 : 5      =     19.5 : 1.0
                  word+2 = 'the'               5 : 2      =     18.1 : 1.0
                  word-2 = 'company'           5 : 6      =     17.7 : 1.0
                  word-1 = 'own'               4 : 6      =     17.4 : 1.0
                  word-1 = 'and'               6 : 5      =     15.8 : 1.0
                  word-2 = 'other'             3 : 6      =     15.3 : 1.0

# Analyse des résultats

## Informations des sens de *interest* :

- Sense 1 =  361 occurrences (15%) - readiness to give attention
- Sense 2 =   11 occurrences (01%) - quality of causing attention to be given to
- Sense 3 =   66 occurrences (03%) - activity, etc. that one gives attention to
- Sense 4 =  178 occurrences (08%) - advantage, advancement or favor
- Sense 5 =  500 occurrences (21%) - a share in a company or business
- Sense 6 = 1252 occurrences (53%) - money paid for the use of money

## Analyse des caractéristiques les plus informatives :

Dans les "most informative features", on compare le ratio de probabilité d'un mot entre un sens et un autre. Voici les observations principales :


1. **`word0 = 'interests'` (89.1:1 pour sens 3 vs 1)** : Le pluriel "interests" indique clairement le sens 3 (activités). Ce qui fait assez sens, on dit "my interests are ..." pour parler d'activités/hobbies.

2. **`word+1 = 'in'` (87.6:1 pour sens 5 vs 6)** : "interest in" favorise fortement le sens 5 (part d'entreprise). Ex: "... interest in the company".

3. **`word+1 = 'of'` (63.2:1 pour sens 4 vs 6)** : "interest of" indique le sens 4 (avantage/intérêt personnel). Ex : "in the interest of...".

4. **`word-1 = '%'` (15.2:1 pour sens 5 vs 6)** : Le symbole % avant "interest" favorise le sens 5 (part d'entreprise) plutôt que 6 (intérêt financier). Ex  : "... 5% interest".

5. **`word-2 = 'company'` (17.7:1 pour sens 5 vs 6)** : "company ... interest" indique clairement le sens 5 (part d'entreprise).

6. **Ponctuation** : Les points (.) et virgules (,) en position +2 favorisent différents sens, intéressant !

7. **`word-1 = 'own'`** : "own interest" favorise le sens 4 (avantage personnel).


**4e.** On souhaite également obtenir les scores pour chaque sens.  Pour ce faire, il faut demander les prédictions une par une au classifieur (voir le [livre NLTK](https://www.nltk.org/book/ch06.html)), et comptabiliser les prédictions correctes pour chaque sens.  Vous pouvez vous inspirer de `evaluate_wsd`, et écrire une fonction `evaluate_wsd_supervised(classifier, items_with_features)`, que vous appliquerez aux donnés `iwf_test`.  Veuillez afficher ces scores.

In [34]:
print(iwf_test_best[:3])

[({'word0': 'interest', 'word-1': 'short', 'word+1': 'dropped', 'word-2': 'that', 'word+2': 'to'}, '5'), ({'word0': 'interest', 'word-1': '}', 'word+1': 'rates', 'word-2': 'NONE', 'word+2': ','}, '6'), ({'word0': 'interests', 'word-1': 'minority', 'word+1': 'rose', 'word-2': 'to', 'word+2': 'to'}, '5')]


In [33]:


def evaluate_wsd_supervised(classifier, items_with_features):

    # Initialiser les compteurs pour chaque sens
    sense_counts = {}  
    sense_correct = {} 
    
    total_correct = 0
    total_instances = 0
    
    for features, true_sense in items_with_features:
        # Extraire le numéro de sens (supprimer /NN, /NNS, etc.)
        true_sense_num = true_sense
        
        # Prédiction du classifieur
        predicted_sense = classifier.classify(features)
        
        # Rajoute le sens dans les compteurs s'il n'existe pas
        if true_sense_num not in sense_counts:
            sense_counts[true_sense_num] = 0
            sense_correct[true_sense_num] = 0
        
        # Compter l'instance
        sense_counts[true_sense_num] += 1
        total_instances += 1
        
        # Vérifier si la prédiction est correcte
        if predicted_sense == true_sense_num:
            sense_correct[true_sense_num] += 1
            total_correct += 1
    
    # Calculer les scores par sens
    sense_scores = {}
    for sense in sorted(sense_counts.keys()):
        if sense_counts[sense] > 0:
            accuracy = sense_correct[sense] / sense_counts[sense]
            sense_scores[sense] = {
                'correct': sense_correct[sense],
                'total': sense_counts[sense],
                'accuracy': accuracy
            }
    
    # Score global pour être sûr d'avoir les mêmes valeurs que plus haut
    global_accuracy = total_correct / (total_instances if total_instances > 0 else 0)
    
    # Affichage des résultats
    print("=== SCORES PAR SENS ===")
    for sense in sorted(sense_scores.keys()):
        stats = sense_scores[sense]
        print(f"Sens {sense}: {stats['correct']}/{stats['total']} = {stats['accuracy']*100:.1f}%")
    
    print(f"\nScore global: {total_correct}/{total_instances} = {global_accuracy*100:.1f}%")
    
    return sense_scores, global_accuracy

# Appliquer la fonction aux données de test
sense_scores, global_acc = evaluate_wsd_supervised(best_classifier, iwf_test_best)



=== SCORES PAR SENS ===
Sens 1: 55/72 = 76.4%
Sens 2: 0/2 = 0.0%
Sens 3: 11/13 = 84.6%
Sens 4: 31/36 = 86.1%
Sens 5: 86/100 = 86.0%
Sens 6: 241/251 = 96.0%

Score global: 424/474 = 89.5%


On voit ici qu'on a pas eu de chance pour le sens 2, mais que les autres sens sont bien prédits. Le sens 6 est très bien prédit, ce qui est logique car il est le plus fréquent et donc il doit être le plus facile à prédire.

## 5. Conclusion

Veuillez recopier ci-dessous, en guise de conclusion, les scores des trois expériences réalisées, pour pouvoir les comparer d'un coup d'oeil.  Quel est le meilleur score obtenu?

In [36]:
import pandas as pd

best_window_lesk_s1 = 16
best_window_lesk_s2 = 18
best_window_w2v_s1 = 12
best_window_w2v_s2 = 13


score_lesk_s1 = evaluate_wsd(wsd_lesk, senses1, sentences, window_size=best_window_lesk_s1)
score_lesk_s2 = evaluate_wsd(wsd_lesk, senses2, sentences, window_size=best_window_lesk_s2)

score_w2v_s1 = evaluate_wsd(wsd_word2vec, senses1, sentences, window_size=best_window_w2v_s1)
score_w2v_s2 = evaluate_wsd(wsd_word2vec, senses2, sentences, window_size=best_window_w2v_s2)

score_classification = nltk.classify.accuracy(classifier, iwf_test) * 100

summary_data = {
    "Méthode": ["Lesk simplifié", "word2Vec", "Classification supervisée"],
    "Score (senses1)": [
        f"{score_lesk_s1:.2f}% (fenêtre {best_window_lesk_s1})",
        f"{score_w2v_s1:.2f}% (fenêtre {best_window_w2v_s1})",
        f"{score_classification:.2f}%"
    ],
    "Score (senses2)": [
        f"{score_lesk_s2:.2f}% (fenêtre {best_window_lesk_s2})",
        f"{score_w2v_s2:.2f}% (fenêtre {best_window_w2v_s2})",
        "N/A"
    ]
}

df_conclusion_student = pd.DataFrame(summary_data)

print("Tableau récapitulatif des scores (Student's Notebook) :\n")
print(df_conclusion_student.to_string(index=False)) # .to_string() pour un meilleur affichage sans index

# Commentaire sur le meilleur score
meilleur_score_global = score_classification # Basé sur les chiffres
print(f"\nLe meilleur score global obtenu dans ce notebook est de {meilleur_score_global:.2f}% avec la méthode de Classification supervisée.")

Tableau récapitulatif des scores (Student's Notebook) :

                  Méthode     Score (senses1)     Score (senses2)
           Lesk simplifié 20.90% (fenêtre 16) 23.61% (fenêtre 18)
                 word2Vec 41.43% (fenêtre 12) 62.42% (fenêtre 13)
Classification supervisée              89.45%                 N/A

Le meilleur score global obtenu dans ce notebook est de 89.45% avec la méthode de Classification supervisée.


## Fin du laboratoire

Merci de nettoyer votre feuille, sauvegarder le résultat, et soumettre le *notebook* sur Cyberlearn.